In [6]:
import os
import sys
import json
import logging
import subprocess
import shutil

In [7]:
def determine_new_sessions(path_to_sessions):
    def has_clu_files(path_to_folder):
        return len([x for x in os.listdir(path_to_folder) if '.clu.' in x]) > 0

    def has_dat_file(path_to_folder):
        return len([x for x in os.listdir(path_to_folder) if x.endswith('.dat')]) > 0

    try:
        sessions = [os.path.join(path_to_sessions, x) for x in os.listdir(path_to_sessions) if os.path.isdir(os.path.join(path_to_sessions, x))]
    except OSError:  # might not exist
        sessions = []

    sessions.sort(reverse=True)

    return [x for x in sessions if has_dat_file(x) and not has_clu_files(x)]

In [8]:
class Spikesorter:
    """
    Spike sorter for a single recording session.
    """
    
    def __init__(self, where, config):
        self.where = where
        self.config = config
        self.verbose = True

    def do_lfp(self):
        os.chdir(self.where)
        res = subprocess.call('ndm_lfp %s' % self.config['filebase'] + ".xml", shell=True)

        if self.verbose:
            print("LFP file created for %s with code %s" % (self.where, str(res)))

    def do_hipass(self):
        os.chdir(self.where)
        res = subprocess.call('ndm_hipass %s' % self.config['filebase'] + ".xml", shell=True)

        if self.verbose:
            print("Hipass completed for %s with code %s" % (self.where, str(res)))

    def do_extractspikes(self):
        os.chdir(self.where)
        res = subprocess.call('ndm_extractspikes %s' % self.config['filebase'] + ".xml", shell=True)

        if self.verbose:
            print("Extractspikes completed for %s with code %s" % (self.where, str(res)))

    def do_pca(self):
        os.chdir(self.where)
        res = subprocess.call('ndm_pca %s' % self.config['filebase'] + ".xml", shell=True)

        if self.verbose:
            print("PCA completed for %s with code %s" % (self.where, str(res)))

    def do_klustakwik(self):
        os.chdir(self.where)
        idxs = [x.split('.')[2] for x in os.listdir(self.where) if '.fet.' in x and not x.startswith('.')]

        #groups_path = os.path.join(self.where, 'groups_with_spikes.csv')
        #if os.path.exists(groups_path):
        #    with open(groups_path, 'r') as f:
        #        unit_groups = [int(x) for x in f.readlines()[0].split(",")]
        #else:
        unit_groups = self.config['unit_groups']

        for i in idxs:
            electrode = int(i)
            if not electrode in unit_groups:  # exclude unit groups
                continue

            num_features = "".join(['1' for x in range(unit_groups[electrode] * 3 + 1)])
            res = subprocess.call('%s %s %s %s %s' % (self.config['path_to_kwik'], \
                   self.config["filebase"], i, self.config["kwik_args"], num_features), shell=True)

            if self.verbose:
                print("Kwik %s completed for %s with code %s" % (str(i), self.where, str(res)))

    def do_clean(self):
        os.chdir(self.where)
        res = subprocess.call('ndm_clean %s' % self.config['filebase'] + ".xml", shell=True)

        if self.verbose:
            print("Cleaning completed for %s with code %s" % (self.where, str(res)))

    def do_backup(self):
        for filename in [x for x in os.listdir(self.where) if '.clu.' in x and not x.endswith('.bak')]:
            target = os.path.join(where, filename + ".bak")
            if not os.path.exists(target):
                shutil.copy(os.path.join(self.where, filename), os.path.join(self.where, filename + ".bak"))

In [9]:
def do_spikesorting(where, config):
    steps = ['lfp', 'hipass', 'extractspikes', 'pca', 'klustakwik', 'clean', 'backup']

    ss = Spikesorter(where, config)
    for step in steps:
        if config['steps'][step]:
            getattr(ss, 'do_%s' % step)()

In [10]:
cfg = {
  "sessions": [
'60_SIT_2023-11-24_17-09-10',
  ],
  "steps": {
    "lfp": False,
    "hipass": True,
    "extractspikes": True,
    "pca": True,
    "klustakwik": True,
    "clean": False,
    "backup": False
  },
  "dry-run": False,
  "processed": "/home/sobolev/nevermind/Andrey/data/",
  #"processed": "/home/sobolev/nevermind/Michael/FreeBehaving/SIT_sessions/",
  "unit_groups": {2: 14, 3: 8, 4: 8, 5: 8, 6: 8}, # unit group: num_of_features, ...
  #"unit_groups": {1: 14, 2: 10, 3: 8, 4: 8, 5: 8, 6: 8}, # unit group: num_of_features, ...
  #"unit_groups": {2: 16}, # unit group: num_of_features, ...
  "path_to_kwik": "/home/sobolev/apps/kkwik/KlustaKwik",
  "kwik_args": "-PenaltyMix 1.0 -MinClusters 85 -MaxClusters 85 -MaxPossibleClusters 200 -UseFeatures",
  "max-sessions": 100
}

# 1111111111111111111111111
# 1111111111111111111111111111111111111111111111111

In [11]:
for session in cfg['sessions']:
    animal = session.split('_')[0]
    where = os.path.join(cfg['processed'], animal, session)
    #where = os.path.join(cfg['processed'], session)
    cfg['filebase'] = os.path.basename(where)
    
    do_spikesorting(where, cfg)


                                                   60_SIT_2023-11-24_17-09-10 - High-pass filtering

Number of channels      = 64
Chunk size (bytes)      = 536870912
Samples/chunk/channel   = 4194304
Window half length      = 16
Input file              = 60_SIT_2023-11-24_17-09-10.dat
Output file             = 60_SIT_2023-11-24_17-09-10.fil

Chunk 1/22 (536870912) ..............................................................################################################################
Chunk 2/22 (536870912) ..............................................................################################################################
Chunk 3/22 (536870912) ..............................................................################################################################
Chunk 4/22 (536870912) ..............................................................################################################################
Chunk 5/22 (536870912) ................................

Iteration 30Q: 54 clusters Score -1.21777e+07 nChanged 1484
Deleting Class 83. Lose 1715.574951 but Gain 5647.625000
Iteration 31Q: 53 clusters Score -1.218319e+07 nChanged 1477
Deleting Class 3. Lose 1723.447144 but Gain 5647.656250
Iteration 32Q: 52 clusters Score -1.218877e+07 nChanged 1468
Deleting Class 58. Lose 1728.811157 but Gain 5647.656250
Iteration 33Q: 51 clusters Score -1.219425e+07 nChanged 1314
Deleting Class 64. Lose 1796.377441 but Gain 5647.625000
Iteration 34Q: 50 clusters Score -1.219954e+07 nChanged 1401
Deleting Class 36. Lose 1823.311279 but Gain 5647.656250
Iteration 35Q: 49 clusters Score -1.220496e+07 nChanged 1306
Deleting Class 17. Lose 1861.682617 but Gain 5647.656250
Iteration 36Q: 48 clusters Score -1.221026e+07 nChanged 1374
Deleting Class 45. Lose 1869.596680 but Gain 5647.640625
Iteration 37Q: 47 clusters Score -1.22154e+07 nChanged 1255
Deleting Class 71. Lose 1867.659424 but Gain 5647.640625
Iteration 38Q: 46 clusters Score -1.222074e+07 nChanged 130

	Iteration 47Q: 3 clusters Score -1111404 nChanged 37
	Iteration 48Q: 3 clusters Score -1111417 nChanged 25
	Iteration 49Q: 3 clusters Score -1111425 nChanged 23
	Iteration 50Q: 3 clusters Score -1111434 nChanged 20
	Iteration 51Q: 3 clusters Score -1111440 nChanged 18
	Iteration 52Q: 3 clusters Score -1111447 nChanged 14
	Iteration 53Q: 3 clusters Score -1111451 nChanged 13
	Iteration 54Q: 3 clusters Score -1111454 nChanged 11
	Iteration 55Q: 3 clusters Score -1111457 nChanged 6
	Iteration 56Q: 3 clusters Score -1111460 nChanged 2
	Iteration 57Q: 3 clusters Score -1111460 nChanged 1
	Iteration 58Q: 3 clusters Score -1111462 nChanged 1
	Iteration 59Q: 3 clusters Score -1111462 nChanged 0
	Iteration 60F: 3 clusters Score -1111462 nChanged 0
Trying to split cluster 11 (437 points) 
	Iteration 0F: 2 clusters Score -54178.7 nChanged 0
	Iteration 0F: 3 clusters Score -51462.71 nChanged 35
	Iteration 1F: 3 clusters Score -51608.77 nChanged 11
	Iteration 2Q: 3 clusters Score -51656.04 nChange

	Iteration 3Q: 3 clusters Score -76631.3 nChanged 1
	Iteration 4Q: 3 clusters Score -76633.38 nChanged 0
	Iteration 5F: 3 clusters Score -76633.38 nChanged 0
Trying to split cluster 29 (1189 points) 
	Iteration 0F: 2 clusters Score -159437.8 nChanged 0
	Iteration 0F: 3 clusters Score -155821.4 nChanged 214
	Iteration 1F: 3 clusters Score -156184.6 nChanged 57
	Iteration 2Q: 3 clusters Score -156295.9 nChanged 36
	Iteration 3Q: 3 clusters Score -156369.3 nChanged 15
	Iteration 4Q: 3 clusters Score -156395.5 nChanged 13
	Iteration 5Q: 3 clusters Score -156420 nChanged 8
	Iteration 6Q: 3 clusters Score -156438.6 nChanged 4
	Iteration 7Q: 3 clusters Score -156447.2 nChanged 3
	Iteration 8Q: 3 clusters Score -156452.6 nChanged 0
	Iteration 9F: 3 clusters Score -156452.6 nChanged 0
Trying to split cluster 31 (1549 points) 
	Iteration 0F: 2 clusters Score -193555.3 nChanged 0
	Iteration 0F: 3 clusters Score -189678.4 nChanged 358
	Iteration 1F: 3 clusters Score -190123.6 nChanged 88
	Iteratio

	Iteration 7Q: 3 clusters Score -507657.9 nChanged 15
	Iteration 8Q: 3 clusters Score -507667.8 nChanged 11
	Iteration 9Q: 3 clusters Score -507674.9 nChanged 4
	Iteration 10Q: 3 clusters Score -507675.5 nChanged 1
	Iteration 11Q: 3 clusters Score -507677.5 nChanged 3
	Iteration 12Q: 3 clusters Score -507679.5 nChanged 3
	Iteration 13Q: 3 clusters Score -507683.2 nChanged 5
	Iteration 14Q: 3 clusters Score -507686.8 nChanged 9
	Iteration 15Q: 3 clusters Score -507693.3 nChanged 8
	Iteration 16Q: 3 clusters Score -507697.8 nChanged 10
	Iteration 17Q: 3 clusters Score -507704.6 nChanged 9
	Iteration 18Q: 3 clusters Score -507710.5 nChanged 6
	Iteration 19Q: 3 clusters Score -507716.6 nChanged 8
	Iteration 20F: 3 clusters Score -507721.2 nChanged 2
	Iteration 21Q: 3 clusters Score -507722.9 nChanged 0
	Iteration 22F: 3 clusters Score -507722.9 nChanged 0
Trying to split cluster 50 (6284 points) 
	Iteration 0F: 2 clusters Score -815034.3 nChanged 0
	Iteration 0F: 3 clusters Score -807962.4

...

	Iteration 25Q: 3 clusters Score -387621.9 nChanged 5
	Iteration 26Q: 3 clusters Score -387622.9 nChanged 1
	Iteration 27Q: 3 clusters Score -387623 nChanged 0
	Iteration 28F: 3 clusters Score -387623 nChanged 0
Trying to split cluster 67 (943 points) 
	Iteration 0F: 2 clusters Score -59836.61 nChanged 0
	Iteration 0F: 3 clusters Score -58357.8 nChanged 265
	Iteration 1F: 3 clusters Score -58614.14 nChanged 69
	Iteration 2F: 3 clusters Score -58695.92 nChanged 37
	Iteration 3Q: 3 clusters Score -58741.91 nChanged 17
	Iteration 4Q: 3 clusters Score -58763.21 nChanged 17
	Iteration 5Q: 3 clusters Score -58781.48 nChanged 12
	Iteration 6Q: 3 clusters Score -58792.96 nChanged 7
	Iteration 7Q: 3 clusters Score -58800.37 nChanged 2
	Iteration 8Q: 3 clusters Score -58803.34 nChanged 1
	Iteration 9Q: 3 clusters Score -58806.31 nChanged 3
	Iteration 10Q: 3 clusters Score -58808.19 nChanged 3
	Iteration 11Q: 3 clusters Score -58813.45 nChanged 1
	Iteration 12Q: 3 clusters Score -58813.81 nChange

	Iteration 26F: 3 clusters Score -138568.6 nChanged 0
Iteration 159Q: 22 clusters Score -4111514 nChanged 6
Iteration 160F: 22 clusters Score -4111520 nChanged 4
Iteration 161Q: 22 clusters Score -4111514 nChanged 5
Iteration 162Q: 22 clusters Score -4111516 nChanged 4
Iteration 163Q: 22 clusters Score -4111516 nChanged 6
Iteration 164Q: 22 clusters Score -4111515 nChanged 7
Iteration 165Q: 22 clusters Score -4111522 nChanged 5
Iteration 166Q: 22 clusters Score -4111519 nChanged 6
Iteration 167Q: 22 clusters Score -4111524 nChanged 8
Iteration 168Q: 22 clusters Score -4111532 nChanged 6
Iteration 169Q: 22 clusters Score -4111534 nChanged 5
Iteration 170Q: 22 clusters Score -4111536 nChanged 5
Iteration 171Q: 22 clusters Score -4111532 nChanged 2
Iteration 172Q: 22 clusters Score -4111532 nChanged 2
Iteration 173Q: 22 clusters Score -4111532 nChanged 2
Iteration 174Q: 22 clusters Score -4111538 nChanged 1
Iteration 175Q: 22 clusters Score -4111540 nChanged 0
Iteration 176F: 22 clusters 

	Iteration 17Q: 3 clusters Score -226883.4 nChanged 19
	Iteration 18Q: 3 clusters Score -226892.3 nChanged 18
	Iteration 19Q: 3 clusters Score -226901.3 nChanged 25
	Iteration 20F: 3 clusters Score -226912.8 nChanged 27
	Iteration 21Q: 3 clusters Score -226926.7 nChanged 20
	Iteration 22Q: 3 clusters Score -226934.8 nChanged 15
	Iteration 23Q: 3 clusters Score -226941.3 nChanged 17
	Iteration 24Q: 3 clusters Score -226948.7 nChanged 20
	Iteration 25Q: 3 clusters Score -226955 nChanged 8
	Iteration 26Q: 3 clusters Score -226956.9 nChanged 9
	Iteration 27Q: 3 clusters Score -226960.6 nChanged 12
	Iteration 28Q: 3 clusters Score -226964.4 nChanged 9
	Iteration 29Q: 3 clusters Score -226967.7 nChanged 6
	Iteration 30Q: 3 clusters Score -226970 nChanged 7
	Iteration 31Q: 3 clusters Score -226972.2 nChanged 6
	Iteration 32Q: 3 clusters Score -226974.1 nChanged 6
	Iteration 33Q: 3 clusters Score -226976.2 nChanged 2
	Iteration 34Q: 3 clusters Score -226976.9 nChanged 3
	Iteration 35Q: 3 clust

	Iteration 91Q: 3 clusters Score -546160.6 nChanged 12
	Iteration 92Q: 3 clusters Score -546164.1 nChanged 12
	Iteration 93Q: 3 clusters Score -546165.7 nChanged 11
	Iteration 94Q: 3 clusters Score -546167.9 nChanged 11
	Iteration 95Q: 3 clusters Score -546169.8 nChanged 10
	Iteration 96Q: 3 clusters Score -546170.2 nChanged 11
	Iteration 97Q: 3 clusters Score -546173.3 nChanged 12
	Iteration 98Q: 3 clusters Score -546174.7 nChanged 5
	Iteration 99Q: 3 clusters Score -546175.2 nChanged 5
	Iteration 100F: 3 clusters Score -546176.6 nChanged 6
	Iteration 101Q: 3 clusters Score -546179.1 nChanged 6
	Iteration 102Q: 3 clusters Score -546180.4 nChanged 5
	Iteration 103Q: 3 clusters Score -546181.1 nChanged 4
	Iteration 104Q: 3 clusters Score -546180.9 nChanged 3
	Iteration 105Q: 3 clusters Score -546182.1 nChanged 0
	Iteration 106F: 3 clusters Score -546182.1 nChanged 0
Trying to split cluster 42 (242 points) 
	Iteration 0F: 2 clusters Score -10294.26 nChanged 0
	Iteration 0F: 3 clusters Sc

	Iteration 4Q: 3 clusters Score -100441.6 nChanged 37
	Iteration 5Q: 3 clusters Score -100471.2 nChanged 33
	Iteration 6Q: 3 clusters Score -100494 nChanged 18
	Iteration 7Q: 3 clusters Score -100506.9 nChanged 25
	Iteration 8Q: 3 clusters Score -100525.8 nChanged 20
	Iteration 9Q: 3 clusters Score -100540.1 nChanged 22
	Iteration 10Q: 3 clusters Score -100556.5 nChanged 21
	Iteration 11Q: 3 clusters Score -100569.2 nChanged 18
	Iteration 12Q: 3 clusters Score -100582 nChanged 14
	Iteration 13Q: 3 clusters Score -100593.9 nChanged 16
	Iteration 14Q: 3 clusters Score -100604.8 nChanged 9
	Iteration 15Q: 3 clusters Score -100610.7 nChanged 9
	Iteration 16Q: 3 clusters Score -100615.9 nChanged 5
	Iteration 17Q: 3 clusters Score -100617.5 nChanged 5
	Iteration 18Q: 3 clusters Score -100619.7 nChanged 5
	Iteration 19Q: 3 clusters Score -100622.5 nChanged 3
	Iteration 20F: 3 clusters Score -100625.7 nChanged 3
	Iteration 21Q: 3 clusters Score -100626.9 nChanged 1
	Iteration 22Q: 3 clusters S

In [1]:
import os

def build_file_path(where, session, file_name):
    return os.path.join(where, session.split('_')[0], session, file_name)

In [31]:
import xml.etree.ElementTree as ET

session = "009266_hippoSIT_2023-04-17_17-04-17"
fp = build_file_path("/home/sobolev/nevermind/Andrey/sandbox/data/", session, "")
xml_path = os.path.join(fp, '%s.xml' % session)
root = ET.parse(xml_path).getroot()

In [32]:
ch_groups_xml = root.findall('spikeDetection')[0].findall('channelGroups')[0]

channel_groups = {}
for i, grp in enumerate(ch_groups_xml):
    for channels in grp:
        if not channels.tag == 'channels':
            continue
        channel_groups[i+1] = [int(ch.text) for ch in channels]

In [34]:
channel_groups

{1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 14, 15],
 2: [17, 18, 19, 20, 22, 23, 24, 25, 27, 28, 29],
 3: [32, 33, 34, 35, 36, 37, 38, 39],
 4: [40, 41, 42, 43, 44, 45, 46, 47],
 5: [48, 49, 50, 51, 52, 53, 54, 55],
 6: [56, 57, 58, 59, 60, 61, 62, 63]}

In [36]:
['%s.spk.%d' % (session, el) for el in channel_groups.keys()]

['009266_hippoSIT_2023-04-17_17-04-17.spk.1',
 '009266_hippoSIT_2023-04-17_17-04-17.spk.2',
 '009266_hippoSIT_2023-04-17_17-04-17.spk.3',
 '009266_hippoSIT_2023-04-17_17-04-17.spk.4',
 '009266_hippoSIT_2023-04-17_17-04-17.spk.5',
 '009266_hippoSIT_2023-04-17_17-04-17.spk.6']

In [42]:
electrodes = list(channel_groups.keys())
electrode = 1

In [49]:
num_fet = ["".join(['1' for x in range(len(channel_groups[el]) * 3 + 1)]) for el in electrodes]

In [46]:
range(len(channel_groups[electrode]) * 3 + 1)

range(0, 43)

In [45]:
channel_groups[electrode]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 14, 15]

In [51]:
[(el, fet) for el, fet in zip(electrodes, num_fet)]

[(1, '1111111111111111111111111111111111111111111'),
 (2, '1111111111111111111111111111111111'),
 (3, '1111111111111111111111111'),
 (4, '1111111111111111111111111'),
 (5, '1111111111111111111111111'),
 (6, '1111111111111111111111111')]